<a href="https://colab.research.google.com/github/Euijune/competition/blob/main/pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**



In [21]:
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets

from torch.utils.data import DataLoader

In [22]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [24]:
print(device)

cuda


In [25]:
!nvidia-smi

Fri Aug 20 08:03:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    58W / 149W |   2644MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
# hyper parameter
learning_rate = 0.001
training_epochs = 40
batch_size = 100

# **Image Resize**

In [27]:
# Compose안에 배치하는 순서조심
trans = transforms.Compose([
  transforms.Resize((64, 128)),  # (height, Width)
  transforms.ToTensor()
])

pokemon_train = datasets.ImageFolder (root='drive/MyDrive/pokemon/train', transform=trans)
#pokemon_test = datasets.ImageFolder (root='drive/MyDrive/pokemon/test', transform=trans)

In [28]:
data_loader = DataLoader(dataset = pokemon_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [29]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
print(len(pokemon_train.classes))   # 분류할 class의 개수

총 배치의 수 : 106
151


# **Model**

In [30]:
#lr = 0.01 kaiming_uniform_ 으로 초기화 시 : cost 평균 4.9
#lr = 0.01 kaiming_normal_ 으로 초기화 시 : cost 평균 4.9
#lr = 0.001일때 cost avg = 0.17

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # 첫번째층
        # ImgIn shape  = (?, 3, 64, 128)  batch size, channel, height, width
        #    Conv     -> (?, 32, 64, 128)
        #    Conv     -> (?, 32, 64, 128)
        #    Pool     -> (?, 32, 32, 64)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 32, 32, 64)
        #    Conv      ->(?, 64, 32, 64)
        #    Conv      ->(?, 128, 32, 64)
        #    Pool      ->(?, 128, 16, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 세번째층
        # ImgIn shape=(?, 128, 16, 32)
        #    Conv      ->(?, 256, 16, 32)
        #    Conv      ->(?, 256, 16, 32)
        #    Pool      ->(?, 256, 8, 16)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층1 256 * 8 * 16 inputs -> 8192 outputs
        self.fc1 = torch.nn.Linear(256 * 8 * 16, 8192, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.kaiming_normal_(self.fc1.weight)

        self.fcLayer1 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        
        # 전결합층2 8192 inputs -> 2048 outputs
        self.fc2 = torch.nn.Linear(8192, 2048, bias=True)
        torch.nn.init.kaiming_normal_(self.fc2.weight)

        self.fcLayer2 = torch.nn.Sequential(
            self.fc2,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))

        # 전결합층3 2048 inputs -> 151 outputs
        self.fc3 = torch.nn.Linear(2048, 151, bias=True)
        torch.nn.init.kaiming_normal_(self.fc3.weight)       

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fcLayer1(out)
        out = self.fcLayer2(out)
        out = self.fc3(out)
        return out

In [31]:
model = CNN().to(device)

In [32]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
# 정확도도 출력이 같이되면 좋을 것 같다.
# validation set 구축 (train에서 떼어오자)

pre_cost = 10000
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (128x64), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

        correct_prediction = torch.argmax(hypothesis, 1) == Y
        

    if avg_cost < pre_cost:
        torch.save(CNN, 'drive/MyDrive/DeepLearning_competition/pokemon_CNN/CNN.pth')
        pre_cost = avg_cost 
    accuracy = correct_prediction.float().mean()
    

    print('[Epoch: {:>4}] cost = {:>.9},    Accuracy = {:>.9}'.format(epoch + 1, avg_cost, accuracy.item()))
    

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


[Epoch:    1] cost = 4.89993238 Accuracy = 0.05999999865889549
[Epoch:    2] cost = 4.57308626 Accuracy = 0.14999999105930328
[Epoch:    3] cost = 4.37740326 Accuracy = 0.10999999940395355
[Epoch:    4] cost = 4.1921463 Accuracy = 0.14999999105930328
[Epoch:    5] cost = 3.9913969 Accuracy = 0.09999999403953552
[Epoch:    6] cost = 3.74804258 Accuracy = 0.12999999523162842
[Epoch:    7] cost = 3.55630565 Accuracy = 0.17000000178813934
[Epoch:    8] cost = 3.31720853 Accuracy = 0.22999998927116394
[Epoch:    9] cost = 3.06053853 Accuracy = 0.26999998092651367
[Epoch:   10] cost = 2.78587055 Accuracy = 0.3199999928474426
[Epoch:   11] cost = 2.50251245 Accuracy = 0.3100000023841858
[Epoch:   12] cost = 2.18536401 Accuracy = 0.3700000047683716
[Epoch:   13] cost = 1.8947705 Accuracy = 0.5099999904632568
[Epoch:   14] cost = 1.63890505 Accuracy = 0.6299999952316284
[Epoch:   15] cost = 1.45018041 Accuracy = 0.5899999737739563
[Epoch:   16] cost = 1.22881329 Accuracy = 0.7099999785423279
[E

In [35]:
model = torch.load('drive/MyDrive/DeepLearning_competition/pokemon_CNN/CNN.pth')

In [ ]:
# test는 custom dataset

with torch.no_grad():
    X_test = pokemon_test.test_data.view(len(pokemon_test), 3, 64, 128).float().to(device)
    # testdata에는 label이 없다. traindata의 label을 가져와야 할듯??
    Y_test = pokemon_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())